In [1]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd
import tensorflow as tf

In [4]:
# Files to load
file_to_load = "Resources/cleaned_us_data.csv"
# Read the CSV into a DataFrame
us_data_df = pd.read_csv(file_to_load)
us_data_df .head()

,Unnamed: 0,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,...,Quantity,Discount,Profit,Shipping Cost,Order Priority,Year,Month_Year,Discount_percentage,Selling Price Per Unit,Cost Price Per Unit
0,7,34662,CA-2011-115161,2011-01-02,2011-03-02,First Class,LC-17050,Liz Carlisle,Consumer,Mission Viejo,...,2,0.15,3.4196,54.64,High,2011,2011-01,15.0,145.333,143.6232
1,27,37844,CA-2011-113880,2011-01-03,2011-05-03,Standard Class,VF-21715,Vicky Freymann,Home Office,Elmhurst,...,6,0.30,-172.1172,70.05,High,2011,2011-01,30.0,105.686,134.3722
2,32,31454,CA-2011-104269,2011-01-03,2011-06-03,Second Class,DB-13060,Dave Brooks,Consumer,Seattle,...,2,0.20,51.4764,47.89,Medium,2011,2011-01,20.0,228.784,203.0458
3,34,39607,CA-2011-168312,2011-01-03,2011-07-03,Standard Class,GW-14605,Giulietta Weimer,Consumer,Houston,...,3,0.30,-43.0296,32.70,Medium,2011,2011-01,30.0,125.503,139.8462
4,36,39245,CA-2011-131009,2011-01-03,2011-05-03,Standard Class,SC-20380,Shahid Collister,Consumer,El Paso,...,6,0.30,0.0000,25.22,Medium,2011,2011-01,30.0,60.375,60.3750


In [18]:
# Generate our categorical variable list
us_cat = us_data_df.dtypes[us_data_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
us_data_df[us_cat].nunique()

Order ID          5009
Order Date        1238
Ship Date         1334
Ship Mode            4
Customer ID        793
Customer Name      793
Segment              3
City               531
State               49
Country              1
Market               1
Region               4
Product ID        1862
Category             3
Sub-Category        17
Product Name      1841
Order Priority       4
Month_Year          48
dtype: int64

In [5]:
us_data_df.columns

Index(['Unnamed: 0', 'Row ID', 'Order ID', 'Order Date', 'Ship Date',
       'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'City', 'State',
       'Country', 'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority', 'Year', 'Month_Year',
       'Discount_percentage', 'Selling Price Per Unit', 'Cost Price Per Unit'],
      dtype='object')

In [29]:
# Define features set
df= us_data_df[["Segment","Category","Profit","Quantity",
            "Shipping Cost", "Sales","Ship Mode","Order Priority"]]
# Generate our categorical variable list
df_cat = df.dtypes[df.dtypes == "object"].index.tolist()
# Check the number of unique values in each column
df[df_cat].nunique()

Segment           3
Category          3
Ship Mode         4
Order Priority    4
dtype: int64

In [30]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[df_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(df_cat)
encode_df.head()

,Segment_Consumer,Segment_Corporate,Segment_Home Office,Category_Furniture,Category_Office Supplies,Category_Technology,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,Ship Mode_Standard Class,Order Priority_Critical,Order Priority_High,Order Priority_Low,Order Priority_Medium
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [31]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(df_cat,1)
df.head()

,Profit,Quantity,Shipping Cost,Sales,Segment_Consumer,Segment_Corporate,Segment_Home Office,Category_Furniture,Category_Office Supplies,Category_Technology,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,Ship Mode_Standard Class,Order Priority_Critical,Order Priority_High,Order Priority_Low,Order Priority_Medium
0,3.4196,2,54.64,290.666,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-172.1172,6,70.05,634.116,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,51.4764,2,47.89,457.568,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,-43.0296,3,32.70,376.509,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0000,6,25.22,362.250,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [32]:
# Remove loan status target from features data
y = df.Quantity
X = df.drop(columns=["Quantity"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.270


In [34]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
235/235 [==============================] - 1s 2ms/step - loss: -28.0575 - accuracy: 0.0894
Epoch 2/50
235/235 [==============================] - 0s 2ms/step - loss: -330.9956 - accuracy: 0.0899
Epoch 3/50
235/235 [==============================] - 0s 2ms/step - loss: -1411.6157 - accuracy: 0.0899
Epoch 4/50
235/235 [==============================] - 0s 2ms/step - loss: -3764.5107 - accuracy: 0.0899
Epoch 5/50
235/235 [==============================] - 0s 2ms/step - loss: -7750.0713 - accuracy: 0.0899
Epoch 6/50
235/235 [==============================] - 0s 2ms/step - loss: -13622.8145 - accuracy: 0.0899
Epoch 7/50
235/235 [==============================] - 1s 2ms/step - loss: -21639.9512 - accuracy: 0.0899
Epoch 8/50
235/235 [==============================] - 1s 3ms/step - loss: -31956.4922 - accuracy: 0.0899
Epoch 9/50
235/235 [==============================] - 1s 3ms/step - loss: -44684.4727 - accuracy: 0.0899
Epoch 10/50
235/235 [==============================] - 1s 6ms/s